# Talking to LDAP servers

To talk to LDAP servers (AD, LDAPS) we depend on the `ldap3` module which needs to be installed first:

```bash
$ pip install --upgrade ldap3
```

To bind to our Active Directory, you need to be within the ETH domain.

In [ ]:
import sys
import getpass
from ldap3 import Server, Connection, ALL, NTLM, SUBTREE, Tls

server = Server("d.ethz.ch", use_ssl=True)
username = input("Username: ")

Now, we can establish a connection. It happens in two steps:

1. connection to the server
2. bind – the actual authentication against the LDAPS server

If the `conn.bind()` is False, then the connection could not be established, e.g. wrong credentials or username

In [ ]:
conn = Connection(
    server, 
    user=f'cn={username},OU=ETHUsers,DC=d,DC=ethz,DC=ch',
    password=getpass.getpass()
)
conn.bind()

**Note:** AD supports NTLM authentication to bind with `d\username` instead of the distinguishedName (dn). Up to now, I could not get it working – if you do know how, please send me a note ;-)

Once we got a successful bind, we can **start searching**

In [ ]:
attributes=["displayName", "mail"]
ok = conn.search(
    search_base="DC=d,DC=ethz,DC=ch",
    search_filter=f"(&(objectClass=user)(sAMAccountName={username}))",
    search_scope=SUBTREE,
    attributes=attributes
)
if ok:
    for response in conn.response:
        if not 'dn' in response: continue
        print(response['dn'])
        for key in response.get('attributes', []):
            print(key, ":", response['attributes'].get(key))

## Exercises

1. Query for groups, using `(&(objectClass=group)(sAMAccountName=id-sis*))`
2. Use the `*` to match any character (only use it at the end, for performance!)
3. Try to use `attributes=["*"]` to get all attributes (watch out, it can be a lot ;-)
4. Query for all computers starting with `Q`, using `(&(objectClass=computer)(sAMAccountName=q*))`, 